In [1]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import urlparse
os.sys.path.append("/home/sparrow/sparrow/malicious_url_cluster")
from config import cfg
from logger import logger

cfg.N_CLUSTER_RATION = 200

cfg.EDIT_DISTANCE_THRESH_LONG = 0.3
cfg.EDIT_DISTANCE_THRESH_SHORT = 0.3
cfg.LONG_URL_THRESH = 100
cfg.SHORT_URL_THRESH = 4

cfg.MIN_SUBSTRING_SIZE = 4
cfg.SINGLE_REGEX_SIZE = 4
cfg.TOTAL_REGEX_SIZE = 4
cfg.SINGLE_REGEX_SIZE_RATIO = 0.2
cfg.TOTAL_REGEX_SIZE_RATIO = 0.2

cfg.DOMAIN_CLUSTER_SIZE_THRESH = 3

import preprocess
import vectorize
import clustering
import regex_url
import urlnormalize
import regex_domain
from file_io import load_urls, dump_urls

## Prepare Testing Set

In [8]:
def get_malware_edr_data(file_list):
    root_path = "/home/sparrow/sparrow/data/sangfor/18"
    df_list = []
    for i in file_list:
        local_path = root_path + i
        filepath = os.path.join(local_path ,'dumped_result.edr.csv')
        df_list.append( pd.read_csv(filepath))
        print local_path
        
    df = pd.concat(df_list, axis = 0)
    return df

file_list = ["0401", "0402", "0410", "0411", "0412", "0415", "0416", "0417", "0418", "0419"]
df = get_malware_edr_data(file_list)
df = df.drop_duplicates("url")
urls = list(df.url)
print "\nEDR total count: %d\n" % len(df)

url_0 = list(df.loc[df.attr == 0].url)
url_1 = list(df.loc[df.attr == 1].url)
url_2 = list(df.loc[df.attr == 2].url)
url_3 = list(df.loc[df.attr == 3].url)
print "0 malware\t%d\n1 benign\t%d\n2 unknown\t%d\n3 possible benign\t%d" %(len(url_0), len(url_1), len(url_2), len(url_3))
df = df.set_index("url")

/home/sparrow/sparrow/data/sangfor/180401
/home/sparrow/sparrow/data/sangfor/180402
/home/sparrow/sparrow/data/sangfor/180410
/home/sparrow/sparrow/data/sangfor/180411
/home/sparrow/sparrow/data/sangfor/180412
/home/sparrow/sparrow/data/sangfor/180415
/home/sparrow/sparrow/data/sangfor/180416
/home/sparrow/sparrow/data/sangfor/180417
/home/sparrow/sparrow/data/sangfor/180418
/home/sparrow/sparrow/data/sangfor/180419

EDR total count: 36073

0 malware	2801
1 benign	296
2 unknown	32972
3 possible benign	4


## Prepare Trainning Data

In [7]:
sangfor_url = list(set(load_urls("../../data/sangfor/malware_all_0419.csv", csv = True)))
vt_url_su = list(set(load_urls("../../data/VirusTotalnew/malicious/180421-ma.txt", csv = False)))
vt_url_ma = list(set(load_urls("../../data/VirusTotalnew/malicious/180421-su.txt", csv = False)))
train_urls = list(set(sangfor_url + vt_url_su + vt_url_ma) )
print len(train_urls)

2018-04-21 10:53:23 file_io.py [line:52] DEBUG	URLs Count:	71201
2018-04-21 10:53:23 file_io.py [line:52] DEBUG	URLs Count:	104525
2018-04-21 10:53:23 file_io.py [line:52] DEBUG	URLs Count:	850584


994836


In [20]:
dump_urls(train_urls, "../../data/EXP_domain/malware.csv")

2018-04-21 11:25:39 file_io.py [line:33] DEBUG	URLs has been dump	../../data/EXP_domain/malware.csv


In [8]:
url_domain, url_path = preprocess.make_url_split(train_urls)
print len(url_domain)
print len(url_path)
urls = list(url_domain)

360770
519300


In [12]:
clustering.make_kmeans_cluster_mass?

In [13]:
cc_list = clustering.make_kmeans_cluster_mass(urls,domain = True, path = False, param = False)

2018-04-21 10:58:05 vectorize.py [line:77] DEBUG	vectorization complete! data shape:	(360770, 128)
2018-04-21 10:58:05 clustering.py [line:224] DEBUG	Begin to make k-means cluster, data_size: 360770 n_clusters: 180
2018-04-21 10:58:05 clustering.py [line:245] DEBUG	optimal n_cluster	7
2018-04-21 10:58:40 clustering.py [line:263] DEBUG	k-means cluster done!
2018-04-21 10:58:40 clustering.py [line:291] DEBUG	Preliminary K-means clustering complete
2018-04-21 10:58:44 vectorize.py [line:77] DEBUG	vectorization complete! data shape:	(50867, 128)
2018-04-21 10:58:44 clustering.py [line:224] DEBUG	Begin to make k-means cluster, data_size: 50867 n_clusters: 25
2018-04-21 10:58:44 clustering.py [line:245] DEBUG	optimal n_cluster	3
2018-04-21 10:58:50 clustering.py [line:263] DEBUG	k-means cluster done!
2018-04-21 10:58:56 vectorize.py [line:77] DEBUG	vectorization complete! data shape:	(98049, 128)
2018-04-21 10:58:56 clustering.py [line:224] DEBUG	Begin to make k-means cluster, data_size: 980

In [14]:
clustering.make_string_distance_cluster(data = cc_list, 
                            metric = "distance",  
                            file_path = "../../data/EXP_domain/cluster_distance.json", )

index 0 0/41535
index 0 0/35683
index 0 0/35620
index 0 0/30939
index 0 0/26746
index 0 0/26739
index 0 0/25817
index 0 0/19861
index 100 100/19861
index 100 100/30939
index 100 100/26746
index 100 100/35620
index 100 100/35683
index 200 200/19861
index 200 200/30939
index 100 100/25817
index 300 300/30939
index 200 200/26746
index 400 400/19861
index 200 200/35620
index 200 200/35683
index 100 100/26739
index 200 200/25817
index 900 900/41535
index 300 300/26746
index 500 500/30939
index 600 600/19861
index 300 300/35620
index 700 700/19861
index 600 600/30939
index 300 300/25817
index 1300 1300/41535
index 400 400/26746
index 800 800/19861
index 700 700/30939
index 900 900/19861
index 400 400/35620
index 1600 1600/41535
index 200 200/26739
index 400 400/35683
index 800 800/30939
index 500 500/26746
index 1000 1000/19861
index 400 400/25817
index 900 900/30939
index 1200 1200/19861
index 500 500/35620
index 500 500/35683
index 600 600/26746
index 1000 1000/30939
index 1300 1300/19861


index 39800 39800/41535
index 39900 39900/41535
index 3900 3900/35683
index 40800 40800/41535
index 41000 41000/41535
index 41200 41200/41535
index 41400 41400/41535
index 9400 9400/30939
index 0 0/19303
index 1500 1500/19656
index 3900 3900/35620
index 5000 5000/26746
index 9500 9500/30939
index 100 100/19303
index 1600 1600/19656
index 2200 2200/26739
index 4000 4000/35683
index 5100 5100/26746
index 200 200/19303
index 4000 4000/35620
index 9900 9900/30939
index 4900 4900/25817
index 5200 5200/26746
index 4100 4100/35683
index 10000 10000/30939
index 400 400/19303
index 10100 10100/30939
index 2000 2000/19656
index 10200 10200/30939
index 5300 5300/26746
index 2100 2100/19656
index 10300 10300/30939
index 4200 4200/35683
index 5100 5100/25817
index 2200 2200/19656
index 600 600/19303
index 10400 10400/30939
index 5400 5400/26746
index 4200 4200/35620
index 10500 10500/30939
index 700 700/19303
index 10600 10600/30939
index 5500 5500/26746
index 4300 4300/35683
index 800 800/19303
in

index 27200 27200/30939
index 15600 15600/19303
index 27300 27300/30939
index 27400 27400/30939
index 15800 15800/19303
index 27500 27500/30939
index 27600 27600/30939
index 27700 27700/30939
index 16100 16100/19303
index 27800 27800/30939
index 7200 7200/35620
index 27900 27900/30939
index 16300 16300/19303
index 28000 28000/30939
index 10400 10400/25817
index 16400 16400/19303
index 28100 28100/30939
index 28200 28200/30939
index 9900 9900/26746
index 1300 1300/17140
index 16600 16600/19303
index 16700 16700/19303
index 28400 28400/30939
index 28600 28600/30939
index 17000 17000/19303
index 28700 28700/30939
index 28800 28800/30939
index 28900 28900/30939
index 10500 10500/25817
index 29100 29100/30939
index 29400 29400/30939
index 18000 18000/19303
index 18100 18100/19303
index 29700 29700/30939
index 29800 29800/30939
index 29900 29900/30939
index 30000 30000/30939
index 18700 18700/19303
index 30300 30300/30939
index 30400 30400/30939
index 30500 30500/30939
index 30600 30600/3093

index 19300 19300/25817
index 2700 2700/11547
index 5900 5900/14661
index 9900 9900/17140
index 19400 19400/25817
index 15400 15400/26746
index 10000 10000/17140
index 2900 2900/11547
index 10100 10100/17140
index 10400 10400/35683
index 6100 6100/14661
index 19700 19700/25817
index 10300 10300/35620
index 3100 3100/11547
index 6300 6300/26739
index 3200 3200/11547
index 3300 3300/11547
index 6300 6300/14661
index 15700 15700/26746
index 20100 20100/25817
index 10500 10500/35683
index 3500 3500/11547
index 10400 10400/35620
index 20300 20300/25817
index 15800 15800/26746
index 11100 11100/17140
index 3700 3700/11547
index 20500 20500/25817
index 11400 11400/17140
index 3800 3800/11547
index 15900 15900/26746
index 11500 11500/17140
index 3900 3900/11547
index 6400 6400/26739
index 11600 11600/17140
index 20700 20700/25817
index 10500 10500/35620
index 6700 6700/14661
index 16000 16000/26746
index 4100 4100/11547
index 20900 20900/25817
index 21000 21000/25817
index 6800 6800/14661
inde

index 6500 6500/7910
index 6600 6600/7910
index 6800 6800/7910
index 7000 7000/7910
index 13600 13600/35620
index 7400 7400/7910
index 7800 7800/7910
index 7900 7900/7910
index 13800 13800/35683
index 8500 8500/26739
index 13900 13900/35683
index 13800 13800/35620
index 14000 14000/35620
index 14200 14200/35683
index 8800 8800/26739
index 14100 14100/35620
index 14200 14200/35620
index 14400 14400/35683
index 8900 8900/26739
index 14300 14300/35620
index 14500 14500/35683
index 14400 14400/35620
index 9000 9000/26739
index 14500 14500/35620
index 14700 14700/35683
index 14600 14600/35620
index 14800 14800/35683
index 14900 14900/35620
index 15000 15000/35683
index 15000 15000/35620
index 15100 15100/35683
index 9400 9400/26739
index 8800 8800/11329
index 15200 15200/35620
index 15300 15300/35683
index 15300 15300/35620
index 15400 15400/35683
index 9500 9500/26739
index 10000 10000/11329
index 15400 15400/35620
index 15600 15600/35683
index 15600 15600/35620
index 9700 9700/26739
index

index 35100 35100/35620
index 35300 35300/35620
index 35500 35500/35620
index 35600 35600/35620
index 32000 32000/35683
index 32100 32100/35683
index 18300 18300/26739
index 32300 32300/35683
index 32400 32400/35683
index 18400 18400/26739
index 32600 32600/35683
index 32700 32700/35683
index 32800 32800/35683
index 32900 32900/35683
index 33000 33000/35683
index 18500 18500/26739
index 33100 33100/35683
index 33200 33200/35683
index 33300 33300/35683
index 33400 33400/35683
index 33500 33500/35683
index 18600 18600/26739
index 33700 33700/35683
index 33800 33800/35683
index 33900 33900/35683
index 34000 34000/35683
index 34100 34100/35683
index 34300 34300/35683
index 34500 34500/35683
index 34600 34600/35683
index 34700 34700/35683
index 34800 34800/35683
index 34900 34900/35683
index 35000 35000/35683
index 35100 35100/35683
index 35200 35200/35683
index 35300 35300/35683
index 35500 35500/35683
index 18800 18800/26739
index 18900 18900/26739
index 19400 19400/26739
index 19600 1960

2018-04-21 11:16:17 clustering.py [line:109] DEBUG	string distance time cost:	923.536451


In [18]:
regex_list = regex_domain.domain_regex_extract(
    input_file_path = "../../data/EXP_domain/cluster_distance.json",
    output_file_path = "../../data/EXP_domain/regex_raw.txt",
    cluster_size_thresh=6)

2018-04-21 11:23:09 file_io.py [line:172] DEBUG	Cluster Data has been loaded	../../data/EXP_domain/cluster_distance.json
2018-04-21 11:23:09 regex_domain.py [line:319] DEBUG	total cluster num:	185589
2018-04-21 11:23:09 regex_domain.py [line:321] DEBUG	big cluster:	12287
2018-04-21 11:23:09 regex_domain.py [line:323] DEBUG	small cluster:	31742
2018-04-21 11:23:09 regex_domain.py [line:324] DEBUG	single one:	141560
2018-04-21 11:23:09 regex_domain.py [line:325] DEBUG	cluster size detail:	Counter({1: 141560, 2: 20441, 3: 7428, 4: 3873, 5: 2394, 6: 1660, 7: 1222, 8: 956, 9: 758, 10: 621, 11: 539, 12: 415, 13: 385, 14: 301, 15: 234, 16: 234, 17: 202, 18: 202, 19: 149, 21: 118, 20: 113, 22: 96, 23: 81, 25: 70, 24: 66, 26: 53, 31: 51, 27: 47, 28: 46, 30: 44, 33: 39, 29: 37, 32: 33, 34: 32, 35: 28, 39: 28, 38: 27, 37: 25, 36: 24, 45: 23, 50: 22, 41: 21, 40: 18, 44: 18, 57: 17, 42: 16, 51: 16, 43: 15, 47: 15, 52: 15, 61: 15, 70: 14, 46: 13, 58: 13, 63: 13, 66: 13, 67: 13, 74: 13, 64: 12, 56: 1

2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nljhh\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nljhh'], 2: ['\\w{3}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3}\.nljhh\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nhdny'], 2: ['\\w{3,6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nhdny\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'llhrx'], 2: ['\\w{2,5}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{2,5}\.llhrx\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'pttfy'], 2: ['\\w{3}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3}\.pttfy\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'faith'], 1: [u'vnhu'], 2: [u'99d9n'], 3: ['\\w{

2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{2,5}\.llhrx\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'krwrn'], 2: ['\\w{3,5}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,5}\.krwrn\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'051intn'], 2: [u'q']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^q\.051intn\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'npxvb'], 2: ['\\w{2,5}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{2,5}\.npxvb\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'rygdd'], 2: ['\\w{3}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3}\.rygdd\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'llhrx'], 2: ['\\w{3,5}']})
2018-04-21 1

2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,5}\.npxvb\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'npxvb'], 2: ['\\w{3,5}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,5}\.npxvb\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'krwrn'], 2: ['\\w{3,5}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,5}\.krwrn\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'knyxj'], 2: ['\\w{3}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3}\.knyxj\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nljhh'], 2: ['\\w{3,6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nljhh\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'97sky'], 2: ['\\w{3,4}']})
2018

2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nhklg'], 2: ['\\w{3,6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nhklg\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nhdny'], 2: ['\\w{3,6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nhdny\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nhdny'], 2: ['\\w{3,6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nhdny\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'nhdny'], 2: ['\\w{3,6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{3,6}\.nhdny\.cn$
2018-04-21 11:23:13 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'win'], 1: [u'fnkjf'], 2: [u'ckq02'], 3: ['\\w{6}']})
2018-04-21 11:23:13 regex_domain.py [line:299] DEBUG	^\w{6}\

...

2018-04-21 11:23:32 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'faith'], 1: [u'vjde'], 2: [u'www'], 3: [u'thvgl'], 4: [u'q81huf'], 5: [u'0gdind'], 6: ['\\w{4,6}']})
2018-04-21 11:23:32 regex_domain.py [line:299] DEBUG	^\w{4,6}\.0gdind\.q81huf\.thvgl\.www\.vjde\.faith$
2018-04-21 11:23:32 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'k2jis'], 2: [u'www'], 3: [u'pm2zj'], 4: ['\\w{6}']})
2018-04-21 11:23:32 regex_domain.py [line:299] DEBUG	^\w{6}\.pm2zj\.www\.k2jis\.download$
2018-04-21 11:23:32 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'x4yil'], 2: [u'www'], 3: [u'5dr6f'], 4: ['\\w{4,6}'], 5: ['\\w{5,6}']})
2018-04-21 11:23:32 regex_domain.py [line:299] DEBUG	^\w{5,6}\.\w{4,6}\.5dr6f\.www\.x4yil\.download$
2018-04-21 11:23:32 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'faith'], 1: [u'josid'], 2: [u'iu22q'], 3: [u'21026'], 4: [u'y5fekp'], 5: [u'0snod4', u'huf4rn']

2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\w{5,6}\.1mab9\.www\.8ibv1\.download$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'k2jis'], 2: [u'www'], 3: [u'pm2zj'], 4: [u'85051'], 5: ['\\w{5,6}']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\w{5,6}\.85051\.pm2zj\.www\.k2jis\.download$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'win'], 1: [u'478sk'], 2: [u'www'], 3: [u'6ndldt'], 4: [u'62469'], 5: ['\\w{6}'], 6: ['\\w{6}']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\w{6}\.\w{6}\.62469\.6ndldt\.www\.478sk\.win$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'win'], 1: [u'dichanpinglun'], 2: [u'www'], 3: [u'krq1e'], 4: [u'bmztor'], 5: [u'92862'], 6: [u'44782', u'z4djf8', u'3l8y8x', u'9f4gal']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^(:?44782|z4djf8|3l8y8x|9f4gal)\.92862\.bmztor\.krq1e\.www\.dichanpinglu

2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'2r40f'], 2: [u'www'], 3: [u'26cd5'], 4: [u'9kbb4n', u'53227', u'62505', u'o4ej0p'], 5: [u'o8cd0v', u'26403', u'dzv5lh', u'81994']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^(:?o8cd0v|26403|dzv5lh|81994)\.(:?9kbb4n|53227|62505|o4ej0p)\.26cd5\.www\.2r40f\.download$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'ar9r6'], 2: [u'www'], 3: [u'09167'], 4: ['\\w{6}'], 5: ['\\w{5,6}']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\w{5,6}\.\w{6}\.09167\.www\.ar9r6\.download$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'k2jis'], 2: [u'www'], 3: [u'8vjai'], 4: [u'45176', u'4evc8c', u'83680', u'78284'], 5: [u'17640', u'rxtw72', u'14953', u'9tqgfe']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^(:?17640|rxtw72|14953|9tqgfe)\.(:?45176|4evc8c|83680|7

2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'q3537'], 2: [u'www'], 3: [u'dllm3'], 4: [u'o147im'], 5: ['\\w{5,6}']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\w{5,6}\.o147im\.dllm3\.www\.q3537\.download$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'k8pnq'], 2: [u'www'], 3: [u'8wetiv'], 4: [u'cs', u'v3sc9e'], 5: ['\\w{5,6}']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\w{5,6}\.(:?cs|v3sc9e)\.8wetiv\.www\.k8pnq\.download$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'cn'], 1: [u'sx'], 2: [u'adsl-pool'], 3: [u'118'], 4: [u'81'], 5: ['\\d{1,3}'], 6: ['\\d{2,3}']})
2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^\d{2,3}\.\d{1,3}\.81\.118\.adsl-pool\.sx\.cn$
2018-04-21 11:23:33 regex_domain.py [line:298] DEBUG	defaultdict(<type 'list'>, {0: [u'download'], 1: [u'c1rdu'], 2: [u'85543'], 3: [u'tl9l5z'], 4: 

2018-04-21 11:23:33 regex_domain.py [line:299] DEBUG	^[^\.]{62}\.(:?0-0-0-0-0-0-0-0-0-0-0-0-0-7-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-59-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-41-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-45-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-53-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-38-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-33-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-47-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-56-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-60-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-28-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-44-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-23-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-46-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-57-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-19-0-0-0-0-0-0-0-0-0-0-0-0-0|0-0-0-0-0-0-0-0-0-0-0-0-0-58-0-0-0-0-0-0-0-0-0-0-0-0-

In [2]:
df = regex_domain.domain_regex_check(
    input_file_path          = "../../data/EXP_domain/regex_raw.txt", 
    test_benign_file_path    = "../../data/sangfor/safe_0419.csv",
    test_malicious_file_path = "../../data/EXP_domain/malware.csv",
    result_file_path         = "../../data/EXP_domain/regex_result.txt", 
    n_jobs = 8)

2018-04-21 11:58:14 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_domain/regex_raw.txt
2018-04-21 11:58:16 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/safe_0419.csv
2018-04-21 11:58:18 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/EXP_domain/malware.csv
2018-04-21 11:58:18 regex_domain.py [line:408] DEBUG	regex count: 4586
2018-04-21 11:58:18 regex_domain.py [line:409] DEBUG	benign urls count: 1461039
2018-04-21 11:58:18 regex_domain.py [line:410] DEBUG	malicious urls count: 994836


batch index 0	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 9	TP(domain) 8	^\w{4,6}\.tti41k\.64ryqf\.9s40r\.www\.vird\.win$
batch index 0	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{6}\.p9fev\.www\.fgi5\.win$
batch index 0	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 12	TP(domain) 12	^\w{6}\.wuo6c\.f8d9\.faith$
batch index 1	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 38	TP(domain) 30	^\w{5,6}\.8we8w\.fkls\.faith$
batch index 0	sample index 3	FP(url) 0	FP(domain) 0	TP(url) 29	TP(domain) 4	^\w{4,6}\.\w{5,6}\.47160\.ftt\.(:?y3mx35|11xpwb)\.wymigx\.zjwpq1\.kskddh\.co0qvm\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 1	sample index 1	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 14	^\d{2,3}\.(:?003096|003097)\.com$
batch index 0	sample index 4	FP(url) 9247	FP(domain) 3161	TP(url) 249	TP(domain) 98	^www\.[A-Za-z]{4,8}\.com\.cn$
batch index 1	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 14	^\w{5,6}\.37l13\.dsnjfk\.win$
batch index 2	sample index 0	

batch index 6	sample index 2	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^nowinstallupgradefreeforsystemssetpcandmac\.win$
batch index 1	sample index 14	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^yourbestandfinest4upgradingalways\.(:?bid|top)$
batch index 4	sample index 7	FP(url) 0	FP(domain) 0	TP(url) 24	TP(domain) 17	^\w{5,6}\.m4s2o\.jkdf\.faith$
batch index 3	sample index 10	FP(url) 0	FP(domain) 0	TP(url) 28	TP(domain) 24	^\w{5,6}\.o4ccu\.ajio\.win$
batch index 7	sample index 0	FP(url) 0	FP(domain) 0	TP(url) 9	TP(domain) 7	^\w{3,6}\.6219y\.vqlln\.download$
batch index 0	sample index 17	FP(url) 0	FP(domain) 0	TP(url) 24	TP(domain) 22	^[A-Za-z]{5,8}\.en\.world-stone\.com$
batch index 5	sample index 5	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^\w{5,6}\.bhc6h\.www\.vrfd\.faith$
batch index 2	sample index 13	FP(url) 0	FP(domain) 0	TP(url) 30	TP(domain) 28	^\w{4,6}\.t8eon\.mu3y4\.download$
batch index 6	sample index 3	FP(url) 0	FP(domain) 0	TP(url) 23	TP(domain) 19	^\w{5,6}\.p77

batch index 2	sample index 21	FP(url) 0	FP(domain) 0	TP(url) 21	TP(domain) 4	^\w{5,6}\.47160\.(:?www|ftt)\.11xpwb\.wymigx\.zjwpq1\.kskddh\.co0qvm\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 4	sample index 16	FP(url) 0	FP(domain) 0	TP(url) 12	TP(domain) 11	^\w{5,6}\.d6793\.www\.itor\.win$
batch index 6	sample index 11	FP(url) 1	FP(domain) 1	TP(url) 19	TP(domain) 6	^down\w{0,6}\.52zsoft\.com$
batch index 1	sample index 23	FP(url) 0	FP(domain) 0	TP(url) 12	TP(domain) 10	^\w{5,6}\.7j88p\.www\.fjvru8\.win$
batch index 3	sample index 19	FP(url) 0	FP(domain) 0	TP(url) 16	TP(domain) 15	^\w{5,6}\.3rb73\.fiei3\.win$
batch index 0	sample index 26	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^ask24updateultimateformacandpc\.(:?info|win|top)$
batch index 5	sample index 14	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^\d{5}\.vdj15\.vnhu\.faith$
batch index 7	sample index 9	FP(url) 0	FP(domain) 0	TP(url) 13	TP(domain) 12	^\w{5,6}\.www\.478sk\.win$
batch index 2	sample index 22	FP(url) 0	FP(do

batch index 1	sample index 31	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^(:?36126|26050)\.(:?7h03er|2n344q)\.fwc56\.www\.gjoie\.faith$
batch index 0	sample index 34	FP(url) 0	FP(domain) 0	TP(url) 28	TP(domain) 18	^\w{4,6}\.xvft3\.fnjks\.win$
batch index 2	sample index 30	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^\w{5,6}\.www\.7snyj\.download$
batch index 4	sample index 25	FP(url) 0	FP(domain) 0	TP(url) 97	TP(domain) 97	^\w{6,10}\.iw3fhwsvw\.com$
batch index 6	sample index 20	FP(url) 0	FP(domain) 0	TP(url) 18	TP(domain) 18	^\w{5,6}\.kegwc\.ox28e\.download$
batch index 3	sample index 28	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{5,6}\.yowu2\.vtygs\.win$
batch index 5	sample index 23	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 14	^\w{6}\.0e82c\.wg5li\.download$
batch index 7	sample index 18	FP(url) 0	FP(domain) 0	TP(url) 4	TP(domain) 4	^\w{5,6}\.o147im\.dllm3\.www\.q3537\.download$
batch index 1	sample index 32	FP(url) 0	FP(domain) 0	TP(url) 9	TP(domain) 9	^\w{5,6}\.t6i

...

batch index 7	sample index 519	FP(url) 0	FP(domain) 0	TP(url) 20	TP(domain) 17	^\w{5,6}\.8ok2u\.hfusi\.win$
batch index 2	sample index 539	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^thebetterandreliable2updating\.website$
batch index 1	sample index 545	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{5,6}\.xbbbp\.wvk6m\.download$
batch index 5	sample index 537	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{5,6}\.4qwwy\.20s06\.download$
batch index 0	sample index 547	FP(url) 0	FP(domain) 0	TP(url) 52	TP(domain) 52	^\w{3}\.rygdd\.cn$
batch index 4	sample index 540	FP(url) 0	FP(domain) 0	TP(url) 10	TP(domain) 10	^\w{6}\.weww0\.vjjjk\.faith$
batch index 6	sample index 538	FP(url) 0	FP(domain) 0	TP(url) 15	TP(domain) 15	^\w{5,6}\.g42yk\.20s06\.download$
batch index 3	sample index 544	FP(url) 0	FP(domain) 0	TP(url) 0	TP(domain) 0	^www\.rtyszz\.com$
batch index 7	sample index 520	FP(url) 0	FP(domain) 0	TP(url) 12	TP(domain) 9	^\w{5,6}\.2zah4\.www\.vjjg\.faith$
batch index 2	sample

batch index 5	sample index 546	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^theonlygoodplacecontent\.(:?win|bid)$
batch index 1	sample index 554	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 1	^paypal\.com\.my\.cgi\.bin\.webscr\.cmd\.login\.submit\.dispatch\.5885d80a13c0db1f8e263663d3faee8\.signssignssigns\.com$
batch index 2	sample index 548	FP(url) 0	FP(domain) 0	TP(url) 13	TP(domain) 13	^(:?developedsoftforfreeandset|developed-softforfreenow|developedsoftforfreenew|developedsoftforfreenewer)\.(:?pw|club|top|bid)$
batch index 7	sample index 528	FP(url) 0	FP(domain) 0	TP(url) 2849	TP(domain) 2554	^\w{4,6}\.15525\.www\.gtiuo\.win$
batch index 4	sample index 549	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^safeandpreparedupgradesystems\.download$
batch index 3	sample index 553	FP(url) 0	FP(domain) 0	TP(url) 29	TP(domain) 24	^\w{5,6}\.hco6y3\.fhyh4s\.hsx12\.www\.vtygs\.win$
batch index 0	sample index 556	FP(url) 0	FP(domain) 0	TP(url) 14	TP(domain) 13	^\w{5,6}\.6a1ju\.www\.dur34\.win$


batch index 5	sample index 555	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^readyandpreparedsetuptoupdates\.(:?pw|pro)$
batch index 3	sample index 561	FP(url) 0	FP(domain) 0	TP(url) 8	TP(domain) 8	^(:?goodplace-contentsforcentralsitealways|goodplace-contentforcentralsiteclear)\.(:?win|bid|trade|download|date|review)$
batch index 0	sample index 564	FP(url) 0	FP(domain) 0	TP(url) 25	TP(domain) 24	^\w{5,6}\.uquu8\.josid\.faith$
batch index 6	sample index 555	FP(url) 0	FP(domain) 0	TP(url) 2	TP(domain) 2	^24checkforfreesystemslikepcandmacs\.(:?club|pro)$
batch index 2	sample index 557	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^\w{6}\.wuy6z\.www\.vgkj\.faith$
batch index 1	sample index 563	FP(url) 0	FP(domain) 0	TP(url) 67	TP(domain) 58	^\w{5}\.eaajc\.cn$
batch index 7	sample index 537	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^thebigandgreat2upgrades\.win$
batch index 5	sample index 556	FP(url) 0	FP(domain) 0	TP(url) 3	TP(domain) 3	^(:?theserviceforallupdating|theserviceforallupd

batch index 5	sample index 564	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^installupgradesetforpcandmac\.download$
batch index 3	sample index 570	FP(url) 0	FP(domain) 0	TP(url) 19	TP(domain) 19	^\w{4,6}\.ci0a6\.z703h\.download$
batch index 4	sample index 566	FP(url) 0	FP(domain) 0	TP(url) 11	TP(domain) 11	^\w{5,6}\.7hf02\.www\.wipss\.win$
batch index 7	sample index 546	FP(url) 0	FP(domain) 0	TP(url) 12	TP(domain) 12	^\w{5,6}\.979nx\.uewi\.win$
batch index 6	sample index 564	FP(url) 0	FP(domain) 0	TP(url) 7	TP(domain) 7	^\w{6}\.z91j7\.fbhwi6\.win$
batch index 2	sample index 566	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^edu\.nrgof\.cn$
batch index 1	sample index 572	FP(url) 0	FP(domain) 0	TP(url) 56	TP(domain) 38	^\w{5,6}\.fklfd\.faith$
batch index 5	sample index 565	FP(url) 0	FP(domain) 0	TP(url) 18	TP(domain) 18	^\w{5,6}\.i8kq6\.ar9r6\.download$
batch index 4	sample index 567	FP(url) 0	FP(domain) 0	TP(url) 1	TP(domain) 1	^15\.0dzucw\.qmap3\.www\.hugew0\.win$
batch index 3	sampl

2018-04-21 12:40:53 file_io.py [line:255] DEBUG	Check Result has been dump	../../data/EXP_domain/regex_result.txt


In [7]:
df = regex_domain.domain_regex_publish(
                                  "../../data/EXP_domain/regex_result.txt",
                                  "../../data/EXP_domain/regex_publish.txt",
                                  publish_score_thresh = 0.4,
                                  publish_ratio = 10,
                                  publish_fp_thresh = 5000,
                                  publish_tp_thresh = 0)

2018-04-21 12:58:16 file_io.py [line:276] DEBUG	check Data has been loaded	../../data/EXP_domain/regex_result.txt
2018-04-21 12:58:26 regex_domain.py [line:459] DEBUG	regular expression publish	4547
2018-04-21 12:58:26 file_io.py [line:207] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_domain/regex_publish.txt
2018-04-21 12:58:26 file_io.py [line:212] DEBUG	Regex has been dump	../../data/EXP_domain/regex_publish.txt


In [10]:
predict_malicious, predict_dict, predict_dict_detail = regex_domain.malicious_domain_predict(
                      input_file_path = url_2,
                      regex_file_path = "../../data/EXP_domain/regex_publish.txt")

2018-04-21 12:59:25 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_domain/regex_publish.txt


batch index 0	sample index 0	hit domain 0	^\w{4,6}\.tti41k\.64ryqf\.9s40r\.www\.vird\.win$
batch index 0	sample index 1	hit domain 0	^\w{6}\.p9fev\.www\.fgi5\.win$
batch index 0	sample index 2	hit domain 0	^\w{6}\.wuo6c\.f8d9\.faith$
batch index 0	sample index 3	hit domain 0	^\w{4,6}\.\w{5,6}\.47160\.ftt\.(:?y3mx35|11xpwb)\.wymigx\.zjwpq1\.kskddh\.co0qvm\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 0	sample index 4	hit domain 0	^\w{5,6}\.rb733\.a56vr\.download$
batch index 1	sample index 0	hit domain 0	^[A-Za-z]{3,7}up[A-Za-z]{3,5}\.(:?streamingupdatesyouneedtowatchallthismonth|streamingupdatesyou-needtowatchallthismonth)\.(:?review|stream)$
batch index 0	sample index 5	hit domain 0	^\w{6}\.6ihvy\.vjjjk\.faith$
batch index 1	sample index 1	hit domain 0	^\w{4,6}\.izlinix\.com$
batch index 1	sample index 2	hit domain 0	^\w{4,6}\.\w{4,6}\.\w{3,6}\.\w{5,6}\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 0	sample index 6	hit domain 0	^\w{5,6}\.www\.667443\.win$
batc

batch index 3	sample index 15	hit domain 0	^\w{5,6}\.3rb73\.fiei3\.win$
batch index 2	sample index 20	hit domain 0	^\w{5,6}\.www\.wioi\.faith$
batch index 1	sample index 27	hit domain 0	^\w{4,6}\.vlhzp\.fkjjd\.win$
batch index 0	sample index 29	hit domain 0	^\w{4,6}\.mmhtz\.download$
batch index 0	sample index 30	hit domain 0	^2010-kpss\.blogspot\.com\.(:?tr|es)$
batch index 3	sample index 16	hit domain 0	^\w{5,6}\.ow68u\.vkjfir\.win$
batch index 2	sample index 21	hit domain 0	^[A-Za-z]{1,7}\.tiananmenlvyou\.com$
batch index 4	sample index 4	hit domain 0	^\w{6}\.48090\.qyvmj\.www\.jreo\.win$
batch index 4	sample index 5	hit domain 0	^\w{5,6}\.p9znf\.jfjdk\.win$
batch index 3	sample index 17	hit domain 0	^\w{5,6}\.f8fg7\.www\.ruyt\.win$
batch index 2	sample index 22	hit domain 0	^(:?vadvaq|9igl6c|yfs9be|zudew2)\.(:?85065|49686|84937|38485)\.7p733\.dichanpinglun\.win$
batch index 0	sample index 31	hit domain 0	^\w{4,6}\.fuv3e2\.kuis6\.vkjfir\.win$
batch index 1	sample index 28	hit domain

batch index 6	sample index 2	hit domain 0	^\w{6}\.1hubq\.www\.itor\.win$
batch index 1	sample index 42	hit domain 0	^\w{6}\.y6ous\.u487s\.download$
batch index 4	sample index 20	hit domain 0	^\w{6,10}\.iw3fhwsvw\.com$
batch index 2	sample index 37	hit domain 0	^\w{4,6}\.qeiyg\.czwpv\.download$
batch index 6	sample index 3	hit domain 0	^(:?yourbettersofisticatedtoupgrades|yourbettersofisticatedtoupgrading|yourbettersofisticatedtoupgrade)\.(:?website|win|bid|top)$
batch index 3	sample index 33	hit domain 116	^\d{4,8}\.url\.246546\.com$
batch index 1	sample index 43	hit domain 0	^ec\.js\.emitileskl\.com$
batch index 0	sample index 48	hit domain 0	^\w{5,6}\.iim2q\.mi842\.download$
batch index 5	sample index 14	hit domain 0	^\w{6}\.c0s8w\.ajio\.win$
batch index 1	sample index 44	hit domain 0	^ip-[^\.]{3,9}-1[^\.]{1,8}\.ip\.secureserver\.net$
batch index 3	sample index 34	hit domain 0	^\w{5,6}\.u8kwo\.guyg\.win$
batch index 0	sample index 49	hit domain 0	^(:?click2newsafe4update|click2newsaf

batch index 2	sample index 46	hit domain 0	^\w{5,6}\.lfchanraomo\.com$
batch index 4	sample index 33	hit domain 0	^[A-Za-z]{6,8}\.01f9y1y87\.com$
batch index 0	sample index 63	hit domain 0	^\w{4,6}\.www\.vnjeri\.win$
batch index 7	sample index 8	hit domain 0	^\w{5,6}\.www\.478sk\.win$
batch index 6	sample index 13	hit domain 0	^(:?o3s3f6|36996|w05p9m)\.(:?20223|18571|75379)\.qoamw\.ar9r6\.download$
batch index 1	sample index 59	hit domain 0	^\w{5,6}\.2qik8\.vbyd\.win$
batch index 2	sample index 47	hit domain 0	^\w{2,6}\.\w{2,6}\.15525\.www\.gtiuo\.win$
batch index 0	sample index 64	hit domain 0	^\w{6}\.2kpe9\.www\.hf70w\.download$
batch index 3	sample index 47	hit domain 0	^\w{5,6}\.8uig8\.46643\.win$
batch index 1	sample index 60	hit domain 0	^rp\.tiviviv\.com$
batch index 4	sample index 34	hit domain 0	^\w{4,6}\.al2873\.uc28\.xp1t\.sale$
batch index 2	sample index 48	hit domain 0	^pleaseupdatesysformacandpc\.(:?info|pw)$
batch index 6	sample index 14	hit domain 0	^\w{5,6}\.oc5vea\.15

...

batch index 6	sample index 473	hit domain 0	^\w{2,3}\.fjpcty\.com$
batch index 4	sample index 504	hit domain 0	^\w{2,3}\.tsykmy\.com$
batch index 2	sample index 545	hit domain 0	^\w{3,6}\.m66uk\.fghu\.faith$
batch index 3	sample index 525	hit domain 0	^(:?getupgrade|liveupgrade|upgradecircle)\.(:?bestsources4yourtrafficonanyoperatingsystem|bestsources4your-trafficonanyoperatingsystemthismonth|bestsources4your-trafficonanyoperatingsystem|bestsources-4yourtrafficonanyoperatingsystemthismonth)\.(:?date|review)$
batch index 0	sample index 539	hit domain 0	^\w{6,8}\.fqj1s1itp\.com$
batch index 4	sample index 505	hit domain 0	^\w{2,6}\.44573\.0fdgaz\.gif26i\.fmirir\.win$
batch index 5	sample index 495	hit domain 0	^yourgoodcentalforcontentnew\.download$
batch index 0	sample index 540	hit domain 0	^\w{5,6}\.sqyay\.bnuew\.win$
batch index 4	sample index 506	hit domain 0	^\w{5,6}\.r4sz6j\.swdvrq\.15525\.www\.gtiuo\.win$
batch index 7	sample index 484	hit domain 0	^wt\d{2,3}\.downyouxi\.com$
bat

batch index 2	sample index 555	hit domain 0	^\w{6}\.20am4\.fgi5\.win$
batch index 1	sample index 527	hit domain 0	^update\.[A-Za-z]{8,9}\.co\.kr$
batch index 7	sample index 497	hit domain 0	^\w{5,6}\.426ks\.ruyt\.win$
batch index 6	sample index 486	hit domain 0	^brightservicesupdatingsafe\.(:?win|bid)$
batch index 3	sample index 535	hit domain 0	^\w{4,6}\.kk2oe\.7guy5\.download$
batch index 5	sample index 509	hit domain 0	^(:?we188m|8yceq6|sf|nysvp)\.(:?e0gp2l|80mplk)\.zdk75\.www\.xglhc\.faith$
batch index 0	sample index 552	hit domain 0	^\w{5,6}\.uquu8\.josid\.faith$
batch index 1	sample index 528	hit domain 0	^\w{5,6}\.(:?6x2bzj|49601)\.(:?1n0iwd|zoy5zy)\.43762\.6e2x4j\.30w5am\.bn797d\.4ir8yy\.76452\.exea41\.dfahyp\.edu\.cn\.lchongfu\.com$
batch index 2	sample index 556	hit domain 0	^(:?onpefr|aqeplr|d8uetf|lwcpfi)\.(:?8mgitj|6bg63j|ztgttf|em75tq)\.15525\.www\.gtiuo\.win$
batch index 4	sample index 519	hit domain 0	^(:?60hx4k|yxukgj|zcglde|vdn135)\.(:?95976|80191|45975|95971)\.j1nk0\

batch index 4	sample index 530	hit domain 0	^\w{6}\.weww0\.vjjjk\.faith$
batch index 1	sample index 540	hit domain 0	^img\d{1}\.img\.8169\.com$
batch index 3	sample index 548	hit domain 0	^\w{5,6}\.uessu\.gfjir\.win$
batch index 5	sample index 518	hit domain 0	^\w{5,6}\.777rp\.fifjsi\.win$
batch index 7	sample index 508	hit domain 0	^\w{5,6}\.z4qux\.download$
batch index 1	sample index 541	hit domain 0	^\w{4,6}\.nx1rf\.bhjf\.faith$
batch index 4	sample index 531	hit domain 0	^\w{4,6}\.751qk\.www\.vkjfir\.win$
batch index 3	sample index 549	hit domain 0	^\w{2,6}\.6ez7co\.iih9d\.www\.sowrt3\.win$
batch index 5	sample index 519	hit domain 0	^(:?44k1rk|f4l5dn|3eztda)\.(:?69284|75231|79530)\.(:?aqejc9|pqv5zh|svuj1l)\.uul0jd\.dllm3\.www\.q3537\.download$
batch index 7	sample index 509	hit domain 0	^\w{5,6}\.fhj106\.15525\.www\.gtiuo\.win$
batch index 6	sample index 499	hit domain 0	^\w{5,6}\.qw06u\.isdod\.faith$
batch index 3	sample index 550	hit domain 0	^(:?37202|pgrlmx|wv3pwg|svvgwf)\.(:?

batch index 7	sample index 525	hit domain 0	^\w{2,5}\.oumaxite6\.com$
batch index 5	sample index 534	hit domain 0	^(:?5554|moyu|6tvulv)\.(:?9a9n5c|zaopcl)\.t8eon\.mu3y4\.download$
batch index 3	sample index 562	hit domain 0	^\w{4,6}\.87390\.22305\.ytbyzp\.dllm3\.www\.q3537\.download$
batch index 1	sample index 556	hit domain 0	^\w{5,6}\.ac0w6\.wioi\.faith$
batch index 6	sample index 515	hit domain 0	^\w{3}\.nqytc\.cn$
batch index 7	sample index 526	hit domain 0	^(:?pcupgrade|upgradelive|legalsoft)\.(:?centerglobalcontentservice|centerglobal4contentingservice|centerglobal4contentsservice|centerglobaltocontentingservice)\.review$
batch index 5	sample index 535	hit domain 0	^ni1[^\.]{8}\.web\d{2}\.nitrado\.hosting$
batch index 3	sample index 563	hit domain 0	^\w{5,6}\.8c4ai\.smy4o\.download$
batch index 6	sample index 516	hit domain 0	^pmbo7u\.71628\.0d4pv\.www\.0o2dg\.download$
batch index 5	sample index 536	hit domain 0	^\w{5,6}\.czvhfx\.08cwu\.www\.reuif\.faith$
batch index 1	sample in

batch index 4	sample index 566	hit domain 0	^\w{2,4}\.jnlube\.com$
batch index 5	sample index 555	hit domain 0	^118\.(:?123|193)\.116\.200$
batch index 6	sample index 537	hit domain 0	^hostingfree4update\.trade$
batch index 4	sample index 567	hit domain 0	^\w{2,4}\.cache\.qcloudcdn\.com$
batch index 7	sample index 545	hit domain 0	^\d{5}\.vvchem\.com$
batch index 6	sample index 538	hit domain 0	^\w{4,6}\.fifjsi\.win$
batch index 5	sample index 556	hit domain 0	^\w{5,6}\.9ljtz\.fui3\.win$
batch index 7	sample index 546	hit domain 0	^\w{5,6}\.a2s0w\.644099\.win$
batch index 6	sample index 539	hit domain 0	^2014\d{13}\.annengdl\.com$
batch index 5	sample index 557	hit domain 0	^installupgradesetforpcandmac\.download$
batch index 7	sample index 547	hit domain 0	^\w{5,6}\.4y68q\.fxzke\.download$
batch index 6	sample index 540	hit domain 0	^\w{4,6}\.6aaew\.8ibv1\.download$
batch index 5	sample index 558	hit domain 0	^\w{5,6}\.i8kq6\.ar9r6\.download$
batch index 7	sample index 548	hit domain 

In [12]:
len(predict_malicious)

1539

In [26]:
for k, v in predict_dict_detail.iteritems():
    print "%40s\t\tdomain:%d\turl:%d" %(k, len(v), sum([len(_) for _ in v]))

             ^down\w{0,6}\.52zsoft\.com$		domain:1	url:1
               ^bd\d{1,2}\.52lishi\.com$		domain:1	url:2
            ^\d{5}\.url\.9xiazaiqi\.com$		domain:29	url:50
                 ^\w{3,4}\.charrem\.com$		domain:3	url:8
             ^\d{4,5}\.url\.tudown\.com$		domain:1	url:1
               ^\d{4,5}\.url\.7wkw\.com$		domain:7	url:8
    ^[A-Za-z]{2,3}\.mingpinhuishou\.com$		domain:2	url:2
             ^[A-Za-z]{2}\.yunmiss\.net$		domain:1	url:1
          ^\w{4,15}\.000webhostapp\.com$		domain:1	url:1
             ^\d{4,8}\.url\.246546\.com$		domain:116	url:289
    ^[A-Za-z]{5,13}\.000webhostapp\.com$		domain:1	url:1
               ^\w{4,6}\.downyouxi\.com$		domain:2	url:2
            ^\w{3,4}\.kaijiaweishi\.com$		domain:2	url:24
              ^\d{4,5}\.url\.222bz\.com$		domain:36	url:47
                   ^\w{2,3}\.mqego\.com$		domain:10	url:46
          ^(:?xz|c|w|6)\.gyingyuan\.com$		domain:1	url:1
             ^mitaosp\d{3}\.17arch\.com$		domain:5	url:8
               ^\d{5

In [79]:
predict_dict_detail["^\w{2,3}\.mqego\.com$"]

[['http://cy2.mqego.com/weishangguanjia9.0.apk'],
 ['http://c3.mqego.com/soft1/cygwin_setup.zip',
  'http://c3.mqego.com/soft3/audiorecordwizard.zip'],
 ['http://xza.mqego.com/hyxdmxhdlq.zip',
  'http://xza.mqego.com/WinScan2PDF_uc.zip',
  'http://xza.mqego.com/slcbdnnbd.zip',
  'http://xza.mqego.com/portraiture64.zip',
  'http://xza.mqego.com/lastactivityview.zip',
  'http://xza.mqego.com/ccleaner_plus.zip',
  'http://xza.mqego.com/hdtunepro570.zip',
  'http://xza.mqego.com/xposedinstaller.apk'],
 ['http://tcy.mqego.com/dilidili.apk', 'http://tcy.mqego.com/pschouchu.zip'],
 ['http://dx.mqego.com/soft3/dota%20v6.71b%20ai.zip',
  'http://dx.mqego.com/soft1/ceb_yueduqi.zip?filename=ceb_yueduqi.zip',
  'http://dx.mqego.com/xl/usboot.zip?filename=usboot.zip',
  'http://dx.mqego.com/soft1/fiddler2.zip',
  'http://dx.mqego.com/soft1/windowsinstallercleanuputility.zip'],
 ['http://xz1.mqego.com/soft3/winrar_x86_sc.exe',
  'http://xz1.mqego.com/soft/hyper_terminal.zip'],
 ['http://dl8.mqego.co

In [51]:
import validate
reload(validate)
from validate import ValidateWithVT

In [81]:
valid_obj = ValidateWithVT()
df = valid_obj.download_file([_ for i in predict_dict_detail["^\w{2,3}\.mqego\.com$"] for _ in i], 
                             "/home/sparrow/sparrow/data/edr_test/domain2")

2018-04-21 15:21:39 validate.py [line:49] DEBUG	mkdir /home/sparrow/sparrow/data/edr_test/domain2
2018-04-21 15:21:39 validate.py [line:22] DEBUG	index	4	http://xza.mqego.com/WinScan2PDF_uc.zip	01931a6257ed835ee70e7a2fc4644fb93c51d905440dead31acbffb61b7332db
2018-04-21 15:21:39 validate.py [line:22] DEBUG	index	7	http://xza.mqego.com/lastactivityview.zip	26835cee3190a98bf5f96fd7ff58bb1af61fdb7c6856a4ab3bf6390c51c0922b
2018-04-21 15:21:39 validate.py [line:22] DEBUG	index	5	http://xza.mqego.com/slcbdnnbd.zip	f0d0995628d65e7b10ae148cfd4314e357d43effb06733e97c27bccde1cd1289
2018-04-21 15:21:39 validate.py [line:22] DEBUG	index	3	http://xza.mqego.com/hyxdmxhdlq.zip	e1e94b113e746eaf02b196b0c47c7141824ee3f0263da7b283c68412cfdbfd8f
2018-04-21 15:21:40 validate.py [line:22] DEBUG	index	1	http://c3.mqego.com/soft1/cygwin_setup.zip	e94df95e36b50a3f55e2b8df9631e7dc948419632b9b8d59f0a9d30e87fd57a5
2018-04-21 15:21:40 validate.py [line:22] DEBUG	index	9	http://xza.mqego.com/hdtunepro570.zip	1b732de

In [82]:
from collections import Counter
Counter(df["sha256"])

Counter({'003455651e428fdfb102b42e97e5f4faf84feaffa8fe38e94c53e2a10018f3fc': 1,
         '01931a6257ed835ee70e7a2fc4644fb93c51d905440dead31acbffb61b7332db': 1,
         '0a9bbd241f9c4316c86864da7e772ab87fec7b85af645f521de4fe9106a315ba': 1,
         '1b732de57fbb5e06cfd64e4952067f3d00a888ff10c7fdb89be4662193b0ec6b': 1,
         '26835cee3190a98bf5f96fd7ff58bb1af61fdb7c6856a4ab3bf6390c51c0922b': 1,
         '28f33f961cf7f8f0a04b068e18ac3d556eeda0a251789c5705f4b0fc9ceade10': 1,
         '2c11a9996f58ddb150ab4a8b5218c9e299c4481d4ac404fe3d73a1153bad1c81': 1,
         '30b81bf6142418faa7277cf6efd87c0912b6c25bbe25d3aea6c746586e29b5e9': 1,
         '58530552a27fd89215fefe4c05991a507c50570d2990cad998921864c7f274aa': 1,
         '5d585333178f18b68049eec3294adb1deed85275f3b9ea31d96ac57d3ad8ecfe': 1,
         '61481b464d7012d07d6ed38d82a6cd0fa3a00e0c1d01a188aac6ea21f547c773': 1,
         '68f0ee733e67d88268c44b8950ebd98a00f107870961786bef9af770a21aec9f': 1,
         '6c036b8bb0fa920bba4312baaa6901

In [83]:
valid_obj.upload_file_to_vt("/home/sparrow/sparrow/data/edr_test/domain2")

2018-04-21 15:22:27 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-21 15:22:29 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-21 15:22:29 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-21 15:22:32 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-21 15:22:32 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-21 15:22:39 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-21 15:22:39 connectionpool.py [line:824] DEBUG	Starting new

2018-04-21 15:24:38 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-21 15:24:38 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-21 15:24:44 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-21 15:24:44 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-21 15:24:45 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "POST /vtapi/v2/file/scan?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1 HTTP/1.1" 200 580
2018-04-21 15:24:45 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-21 15:24:47 connectionpool.py [line:396] DEBUG	https://www.

[u'e94df95e36b50a3f55e2b8df9631e7dc948419632b9b8d59f0a9d30e87fd57a5',
 u'e1e94b113e746eaf02b196b0c47c7141824ee3f0263da7b283c68412cfdbfd8f',
 u'8cb55bf22b2b6d55da45f73d985d7067ff34ba38e8372046e0637f9fb21e0337',
 u'aa38e4e932f85dd2ae16f88bfef6c730bbec15b37e670e60e4a3925b3cf8ea7d',
 u'77c99878691657a6ddc8a98b4dd3487da54c1cf421cbc880e4e32fc50e14fd32',
 u'a0149c4503b8a1b020cd8f866e86ac4effcbf64acd64fbad7cceaf9db8ac7813',
 u'30b81bf6142418faa7277cf6efd87c0912b6c25bbe25d3aea6c746586e29b5e9',
 u'28f33f961cf7f8f0a04b068e18ac3d556eeda0a251789c5705f4b0fc9ceade10',
 u'8100cba54a4d8e7331efda8e4fb80f6245bbaba680c9ef3375aff799e34d1041',
 u'ec9b1e89291202a413863806cf69b64bb7b68d6e5ad074a623b217ba8be53d5b',
 u'cc8199f2abf5f9d0850b06183d82756065e5070a69e755e52e89bb4f2ded8b46',
 u'aa8e04cd4cf56392c2301154565f273802130d1362c833dc167f7cec815cb679',
 u'c80800c753b09b05233d75e20abbf1173a4832f8e8a9daac9414d9243f2353aa',
 u'94e316c4e5344c13f13218d6d7b44d35fdb6290734f6171abe2efd6ef1253e0f',
 u'26835cee3190a98bf

In [86]:
df = valid_obj.validate_file_with_vt(list(df.sha256))

2018-04-21 15:43:55 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-21 15:43:56 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=5d585333178f18b68049eec3294adb1deed85275f3b9ea31d96ac57d3ad8ecfe HTTP/1.1" 200 None
2018-04-21 15:43:56 validate.py [line:162] DEBUG	index 0	 sha256 5d585333178f18b68049eec3294adb1deed85275f3b9ea31d96ac57d3ad8ecfe	positives 11
2018-04-21 15:44:12 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-21 15:44:13 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=e94df95e36b50a3f55e2b8df9631e7dc948419632b9b8d59f0a9d30e87fd57a5 HTTP/1.1" 200 None
2018-04-21 15:44:13 validate.py [line:162] DEBUG	index 1	 sha256 e94df95e36b50a3f55

2018-04-21 15:48:10 validate.py [line:162] DEBUG	index 15	 sha256 b3fd3381034a0e68416590c1cb47e2f5e1754d79edc0e0a45f1050f73a1e1511	positives 1
2018-04-21 15:48:26 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-21 15:48:27 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=827ecdde1f5f39b2150df4ed6a4434c4bc97373ee90d0d803d90ea98b2168152 HTTP/1.1" 200 None
2018-04-21 15:48:27 validate.py [line:162] DEBUG	index 16	 sha256 827ecdde1f5f39b2150df4ed6a4434c4bc97373ee90d0d803d90ea98b2168152	positives 0
2018-04-21 15:48:43 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-21 15:48:44 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=aa8e04cd4cf56392c2301154

2018-04-21 15:52:41 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=8e47a0190b5f9e4cc8498a6af5fb5cc61337d31e4d5aec9c27e68cbc749bf09c HTTP/1.1" 200 None
2018-04-21 15:52:41 validate.py [line:162] DEBUG	index 31	 sha256 8e47a0190b5f9e4cc8498a6af5fb5cc61337d31e4d5aec9c27e68cbc749bf09c	positives 8
2018-04-21 15:52:57 connectionpool.py [line:824] DEBUG	Starting new HTTPS connection (1): www.virustotal.com
2018-04-21 15:52:58 connectionpool.py [line:396] DEBUG	https://www.virustotal.com:443 "GET /vtapi/v2/file/report?apikey=86328d300ba0498ef2b0ad322ab8c4dd5aa62a1c16ece6946b3f541f0b83ecd1&resource=0a9bbd241f9c4316c86864da7e772ab87fec7b85af645f521de4fe9106a315ba HTTP/1.1" 200 None
2018-04-21 15:52:58 validate.py [line:162] DEBUG	index 32	 sha256 0a9bbd241f9c4316c86864da7e772ab87fec7b85af645f521de4fe9106a315ba	positives 15
2018-04-21 15:53:14 connectionpool.py [line:824]